# Environment Variables

## Cloud Run Test Environment

In [ ]:
# In a terminal
# export NAME=""

In [ ]:
import os

In [ ]:
os.getenv("NAME")

### Install and import libraries

In [ ]:
!pip install streamlit psycopg2-binary -q

In [ ]:
import streamlit as st
import psycopg2

In [ ]:
# Restart the terminal

### app-run

In [ ]:
!mkdir app-run

In [ ]:
cd app-run

In [ ]:
pwd

In [ ]:
%%writefile env.py
# DB Credentials
NAME="matt"

In [ ]:
%%writefile app.py
# Import libraries
import streamlit as st
import psycopg2
import os
import time
from env import *

st.write(os.getenv('Hi'))
st.write(NAME)

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit psycopg2-binary -q
COPY app.py env.py ./app/
EXPOSE 9000
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0","--server.port", "9000"]

### Build the image locally

In [ ]:
!docker build -t app-run . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 9000:9000 -v $(pwd):/app app-run

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
# Remove docker ps
# !docker rm -f <...>

In [ ]:
# !docker rm -f <...>

### Deploy using Cloud Build, Artifact Registry, and Cloud Run

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create app-run \
    --repository-format=docker \
    --location=us-west2 \
    --description="Docker repository"

In [ ]:
%%sh
# Verify
gcloud artifacts repositories list

### Build an image using Dockerfile

In [ ]:
# Check if the path is correct
!pwd

In [ ]:
cd app-run

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --tag us-west2-docker.pkg.dev/$(gcloud config get-value project)/app-run/app-run:tag4

In [ ]:
# It didn't work in Cloud Run.

### Pull the image from Artifact Registry

In [ ]:
# !gcloud auth configure-docker us-west2-docker.pkg.dev

In [ ]:
!docker pull us-west2-docker.pkg.dev/$(gcloud config get-value project)/app-v2/app:tag1

In [ ]:
!docker images

In [ ]:
!docker run -d -p 8501:8501 -v $(pwd):/app us-west2-docker.pkg.dev/$(gcloud config get-value project)/app-v2/app:tag1

In [ ]:
# !docker rm -f <...>